In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from simulator.simulator import Simulator


def keypoints_to_colors(x, y, width, height):
    # Normalize coordinates to range [0, 1]
    x_norm = x / width
    y_norm = y / height

    # Compute hue based on position
    hue = (x_norm + y_norm) / 2  # Simple gradient approach
    # Alternative: hue = (np.arctan2(y_norm - 0.5, x_norm - 0.5) / (2 * np.pi)) % 1

    # Set saturation and value to 1
    hsv = np.stack([hue * 179, np.ones_like(hue) * 255, np.ones_like(hue) * 255], axis=-1).astype(np.uint8)

    # Convert HSV to RGB
    return cv2.cvtColor(hsv[np.newaxis, :, :], cv2.COLOR_HSV2RGB)[0]

In [ ]:
simulator = Simulator(
    blend_file='/home/clementin/Dev/blender-construction/construction_matching.blend',
    points_density=100.0,
    verbose=True
)

In [ ]:
masks, uvs = [], []

for _ in range(16):
    rgb, depth, labels = simulator.render()
    _, _, mask, uv = simulator.compute_point_cloud()
    masks.append(mask)
    uvs.append(uv)
    print(simulator.scene.frame_current)
    plt.imshow(rgb)
    plt.show()
    simulator.step_frame()

uvs = np.stack(uvs)
masks = np.stack(masks)

valid = np.all(masks, axis=0)
uvs = uvs[:, :, valid]

# Blocks of 32x32
_, valid = np.unique(np.int64(uvs[0] / 32), axis=1, return_index=True)
uvs = uvs[:, :, valid]

h, w, _ = rgb.shape
colors = keypoints_to_colors(*uvs[0], w, h) / 255

In [ ]:
plt.imshow(rgb)

for uv, c in zip(uvs.T, colors):
    plt.plot(*uv, c=c, lw=1, alpha=0.3)

plt.scatter(*uvs[-1], c=colors, s=2, alpha=0.7)

In [ ]:
#plt.figure(figsize=(20, 20))
plt.imshow(cv2.imread('flow_end.png')[:, :, ::-1])

for uv, c in zip(uvs.T, colors):
    plt.plot(*(uv * 1), c=c, lw=1, alpha=0.5)

plt.scatter(*(uvs[-1] * 1), c=colors, s=2)
plt.axis('off')
plt.savefig('matching.png', bbox_inches='tight', pad_inches=0, dpi=300)